In [ ]:
using Plots

Источник: https://github.com/villano-lab/galactic-spin-W1

Кривая вращения отображает скорость вращения звезд в галактике с точки зрения их расстояния от центра, также известного как радиус. Используя фотометрические данные светящегося вещества, можно построить кривую вращения галактики. Он используется для оценки массы внутри радиуса путем приравнивания центростремительной силы к силе тяжести:

\begin{equation}
\frac{mv^2}{r}=\frac{G M_{enc}(r)}{r^2}
\end{equation}
<br>
>где:<br>
        $v$ = скорость вращения<br>
        $G$ = гравитационная постоянная<br>
        $M_{enc}(r)$ = масса как функция радиуса<br>
        $r$ = радиус или расстояние от центра галактики
    <br>

Кривые вращения некоторых спиральных галактик обнаруживают несоответствие измеренной и светящейся массы. Связь между скоростью вращения и массой важна для изучения темной материи. Построение кривых вращения спиральных галактик и их компонентов, видимых с ребра, можно использовать в качестве инструмента для нахождения кривой вращения темной материи и соответствующей ей массы.

Этот блокнот призван продемонстрировать три различных типа кривых вращения: вращение твердого тела (фрисби), вращение в виде планет и плоское вращение.

In [ ]:
function MultiplePositions(radiuses, velocities, maxtime, dt)
    
    circouter = 2pi*radiuses[end] # длина внешней орбиты
    T = range(0, step = dt, stop = maxtime)
    # отсекаем время по периоду самой внешней орбиты
    istop = findfirst( t-> velocities[end]*t >= circouter, T )
    Tcut = T[1:istop-2]
    # запоминаем траектории всех тел
    Xs = [ r*cos( v*t/r ) for t in T, (r,v) in zip(radiuses, velocities) ]
    Ys = [ r*sin( v*t/r ) for t in T, (r,v) in zip(radiuses, velocities) ]
    
    return Xs, Ys, Tcut
end;

In [ ]:
function MakeAnimation(radiuses,velocities, maxtime, dt, filename; 
                drawmasses = false, axisunit = "km",
                 masses = 5*ones(length(radiuses)) )
   
    Xs, Ys, T = MultiplePositions(radiuses, velocities, maxtime, dt);
    
    limt = 1.1maximum(radiuses)
    area = ( drawmasses ? log.( [s * 5e-25 for s in masses].+1 ).+1 : masses )
    # ...чтобы кружочки были разного размера
    
    function plotcirc(r) # чтоб рисовать орбиты
        theta = range(0, stop = 2pi, length = 128)
        plot!( r*cos.(theta), r*sin.(theta), line = (1, :black) )
    end

    anim = @animate for (i, t) in enumerate(T)
        scatter( Xs[i,:], Ys[i,:], ms = area, c = :black )

        for r in radiuses
            plotcirc(r)
        end

        plot!( xlim = (-limt,limt), ylim = (-limt,limt), legend = false, size = (300,300), 
             xlab = "x" * axisunit, ylab = "y" * axisunit )
    end
    gif(anim, filename, fps = 20)
end;

In [ ]:
function PlotRotationCurve(radiuses, velocities;
                      xlabel="Radius (km)", ylabel="Velocity (km/s)")
    xl = 0.1maximum(radiuses)
    yl = 0.1maximum(velocities)
    plot( radiuses, velocities, line = (2,:black), m = (5, :black), legend = false, 
            xlim = (0, maximum(radiuses)+xl), 
            ylim = (minimum(velocities)-yl, maximum(velocities)+yl),
            xlab = xlabel, ylab = ylabel, size = (300, 300) )
end;

### Вращение жесткого тела

Самая прямолинейная кривая вращения — это кривая твердого тела, т. е. твердого диска. Скорость вращения в этом случае пропорциональна радиусу вращающегося объекта, который можно обозначить как:

\begin{equation}
v \propto r
\end{equation}

Из-за этого твердое тело имеет кривую вращения, линейно увеличивающуюся с радиусом. Чтобы продемонстрировать это, давайте создадим два массива для радиуса и скорости вращения с именами `radiusRB` и `velocityRB` соответственно:

In [ ]:
radiusRB = [1,2,3,4,5]
velocityRB = [0.1,0.2,0.3,0.4,0.5]

MakeAnimation(radiusRB, velocityRB,   # distance unit to m, velocity unit to m/s
                         100, 1,                        # time and dt
                         "rigid_body.gif", axisunit = ", m") 

In [ ]:
PlotRotationCurve( radiusRB, velocityRB, xlabel = "Radius, m", ylabel = "Velocity, m/s" )

In [ ]:
savefig("rc_rb.png")

При вращении твердого тела все части тела должны сохранять одинаковое положение относительно друг друга на протяжении всего вращения. Другой способ представить это состоит в том, что все части твердого тела имеют одинаковую угловую скорость, то есть скорость изменения их угла относительно горизонтали одинакова.

### Планетарное вращение

Группы тел обладают большей гибкостью при вращении. Наиболее известным примером является модель Кеплера, которая моделирует орбитальное вращение в солнечных системах и движение планет. В этой модели большая часть массы сосредоточена в центре вращающегося объекта, а спутники свободно вращаются вокруг центра. При планетарном вращении скорость вращения обратно пропорциональна квадратному корню из радиуса:

\begin{equation}
v \propto \frac{1}{\sqrt{r}}
\end{equation}

Одним из примеров является наша Солнечная система с восемью планетами, вращающимися вокруг Солнца в центре.

https://nssdc.gsfc.nasa.gov/planetary/factsheet/

In [ ]:
# Parameters
const G = 6.67408e-11         # Gravitational constant (in m^3 kg^-1 s^-2)
const AU = 1.496e11           # AU Astronomical Unit (in meters)
const M_Earth = 5.97e24 # kg

massesSS = [0.0553, 0.815, 1.0, 0.107, 317.8, 95.2, 14.5, 17.1] * M_Earth
radiusSS = [0.387, 0.723, 1.0, 1.52, 5.20, 9.57, 19.17, 30.18] * AU
# Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune
M_Sun = 3.3e5*M_Earth
# Orbital velocity of planets (v) in m/s
velocitySS = sqrt.( accumulate(+, massesSS, init = M_Sun)*G ./ radiusSS );

In [ ]:
velocitySS - [47.4, 35.0, 29.8, 24.1, 13.1, 9.7, 6.8, 5.4]*1000 # сравнение с табличными данными

In [ ]:
MakeAnimation(1e-12*radiusSS, 1e-12*velocitySS,   # change distance unit to km, velocity unit to km/s
                         5e10, 5e7,                        # time and dt
                "solarsystem.gif", drawmasses=true, axisunit = ", ⋅10⁹ km", masses=massesSS) 

In [ ]:
PlotRotationCurve( 1e-12*radiusSS,1e-3*velocitySS,
                  xlabel = "Radius, ⋅10⁹ km", ylabel = "Velocity, km/s" )

In [ ]:
savefig("rc_ss.png")

При планетарном вращении ни одно из тел никак не связано друг с другом, поэтому они могут двигаться относительно друг друга. Часто в центре находится единственный доминирующий источник массы — в случае нашей Солнечной системы Солнце достаточно массивно, чтобы мы могли игнорировать массы планет в наших расчетах. Однако, если между центром и наблюдаемыми нами телами имеется значительное количество массы, мы можем получить другие результаты...

### Ожидаемое и фактическое вращение галактики

http://astroweb.cwru.edu/SPARC/

In [ ]:
SPARC_file_directory = "data/sparc/";
names = readdir(SPARC_file_directory)

In [ ]:
[1:175;][ occursin.("NGC5005", names) ] # ищем конкретную галактику

In [ ]:
using DelimitedFiles

In [ ]:
# Distance = 7.72 Mpc
# Rad	Vobs	errV	Vgas	Vdisk	Vbul	SBdisk	SBbul		
# kpc	km/s	km/s	km/s	km/s	km/s	L/pc^2	L/pc^2

__Header key__:<br>
>_Rad_: radius or distance from the center of galaxy (in kiloparsec) <br>
>_Vobs_: observed velocity/measured datapoints (in km/s) <br>
>_errV_: uncertainty in the observed velocity (in km/s) <br>
>_Vgas_: velocity of the gas component (in km/s) <br>
>_Vdisk_: velocity of the disk component (in km/s) <br>
>_Vbul_: velocity of the bulge component (in km/s) <br>
>_SBdisk_: surface brightness of the disk component (in Luminosity/$ \rm parsec^2$) <br>
>_SBbul_: surface brightness of the bulge component (in Luminosity/$ \rm parsec^2$)

Split columns into arrays and name them according to the header displayed in the cell above.

In [ ]:
Dat = [ readdlm(SPARC_file_directory*name, skipstart = 3) for name in names ];
sz = [ size(d,1) for d in Dat ]
argmax(sz) # в каком файле больше точек

In [ ]:
names[110]

In [ ]:
print( read(SPARC_file_directory*names[110], String) )

In [ ]:
for (i,d) in enumerate(Dat) # где есть балдж
    if sum(d[:,6]) != 0
        print("$i ")
    end
end

In [ ]:
dat = Dat[111] # 14 80 110 74 88 108 anti109 111
(Rs, Vobs, errV, Vgas, Vdisk, Vbul) = [dat[:,i] for i in 1:6 ]

plot( Rs, Vobs, yerr = errV, lab = "V obs", line = (3, :grey), m = (3, :grey, 0.5),
        #background_color = RGB(0.2, 0.2, 0.2),
        title = "UGC03205", xlab = "R, kpc", ylab = "V, km/s")

In [ ]:
savefig("rotcurve_UGC03205.png")

In [ ]:
dt = 1e7*3600*24*365 # переводим скорость в кпк на 10млн лет
kpc2km = 3.0857e+16 
MakeAnimation(Rs[3:12:end], Vobs[3:12:end]/kpc2km*dt, 200, 1,
                         "flatrotation.gif", axisunit = ", kpc") 

In [ ]:
PlotRotationCurve( Rs[3:12:end], Vobs[3:12:end], xlabel = "Radius, kpc", ylabel = "Velocity, km/s" )

>__Total velocity of luminous matter__: <br>
    \begin{equation}
    v_{total,light}(r) = \sqrt{\lvert v_{gas}\rvert v_{gas} + \Upsilon _{bulge} \lvert v_{bulge}\rvert v_{bulge} + \Upsilon _{disk} \lvert v_{disk}\rvert v_{disk}}
    \end{equation}<br>
    
A prefactor or mass-to-light ratio ($\Upsilon$) is added to the disk and the bulge which will be useful when fitting the curve of each component. These prefactors help scaling the curve up and down. You can either change these values manually and see the magnitude of the relevant curve change in the cells below or import the fitting parameters from a python library. <br>

>pref_bulge: bulge prefactor<br>
>pref_disk: disk prefactor

Note that the gas prefactor is fixed. The mass of the gas was calculated assuming a factor of 1.33 to account for the contribution of helium. 

Each component was calculated using the following model: <br>
>Bulge: residual luminosity profile <br>
>Disk: observed [3.6] surface brightness profile<br>
>Gas: H1 surface density profiles or mass models<br>

However, these calculations are beyond the scope of this workshop. 

>Jimenez, Raul, Licia Verde, and S. Peng Oh. **Dark halo properties from rotation curves.** _Monthly Notices of the Royal Astronomical Society_ 339, no. 1 (2003): 243-259. https://doi.org/10.1046/j.1365-8711.2003.06165.x. <br><br>
>Lelli, F., McGaugh, S. S., &amp; Schombert, J. M. (2016). **SPARC: Mass models for 175 disk galaxies with Spitzer photometry and accurate rotation curves.** _The Astronomical Journal_, 152(6), 157. https://doi.org/10.3847/0004-6256/152/6/157 <br><br>
>Matt Newville, Renee Otten, Andrew Nelson, Antonino Ingargiola, Till Stensitzki, Dan Allan, Austin Fox, Faustin Carter, Michał, Ray Osborn, Dima Pustakhod, lneuhaus, Sebastian Weigand, Glenn, Christoph Deil, Mark, Allan L. R. Hansen, Gustavo Pasquevich, Leon Foks, … Arun Persaud. (2021). __lmfit/lmfit-py: 1.0.3 (1.0.3).__ Zenodo. https://doi.org/10.5281/zenodo.5570790. <br><br>
>“Megaparsec: Cosmos.” __Megaparsec__ | _COSMOS_. Swinburne University of Technology. Accessed November 12, 2021. https://astronomy.swin.edu.au/cosmos/m/megaparsec. 
***

In [ ]:
using Optim # https://github.com/JuliaNLSolvers/Optim.jl

In [ ]:
function Vtot(p, r)
    i = findfirst( x-> x==r, Rs ) # or [1:length(Rs);][Rs.==r]
    Vgas[i]^2 + p[1]*Vbul[i]^2 + p[2]*Vdisk[i]^2
end
model(p) = sum( abs2, Vtot.(Ref(p), Rs) - Vobs.^2 );

In [ ]:
res = optimize(model, [1.0, 1.0])

In [ ]:
P = Optim.minimizer(res)

In [ ]:
Vsum = Vtot.(Ref(P), Rs)
 plot( Rs, sqrt.(Vsum), lab = "Vtot", line = (3, :red) )
plot!( Rs, Vobs, yerr = errV, lab = "obs", line = (3, :gray), m = (3,:gray, 0.5) )
plot!( Rs, Vgas,  lab = "gas", line = (2, :blue) )
plot!( Rs, P[2]*Vdisk, lab = "disk", line = (2, :green) )
plot!( Rs, P[1]*Vbul,  lab = "bulge", line = (2, :orange), 
    legend = :right, xlab = "R, kpc", ylab = "V, km/s" )

In [ ]:
savefig("rot_curve_components.png")

In [ ]:
# Для 110 галактики
dt = 1e7*3600*24*365 # переводим скорость в кпк на 10млн лет
kpc2km = 3.0857e+16 
MakeAnimation(Rs[3:12:end], Vsum[3:12:end]/kpc2km*dt, 400, 1,
                         "expected.gif", axisunit = ", kpc") 

In [ ]:
PlotRotationCurve( Rs[3:12:end], Vsum[3:12:end], xlabel = "Radius, kpc", ylabel = "Velocity, km/s" )

>__Velocity__: <br>
    \begin{equation}
    v_{DM}(r) = \sqrt{4 \pi G \rho_{0} r_c^2 \big( 1- \frac{r_c}{r} \arctan{\frac{r}{r_c}}\big)}
    \end{equation}<br>
    where:<br>
        $G$ = gravitational constant<br>
        $\rho_0$ = central mass density (in solar mass/$\rm kpc^3$)<br>
        $r_c$ = core radius (in kpc)<br>

In [ ]:
halo(r,rho0,rc) = 4pi*G*rho0*rc^2 * (1 - rc/r * atan(r/rc)) 
function VwithDM(p, r)
    i = findfirst( x-> x==r, Rs ) # or [1:length(Rs);][Rs.==r]
    Vgas[i]^2 + p[1]*Vbul[i]^2 + 
        p[2]*Vdisk[i]^2 + halo(r, p[3], p[4])
end
model_dm(p) = sum( abs2, VwithDM.(Ref(p), Rs) - Vobs.^2 );

In [ ]:
res = optimize(model_dm, [1.0, 1.0, 1.5e10, 10.0])

In [ ]:
P2 = Optim.minimizer(res)

In [ ]:
Vsum = VwithDM.(Ref(P2), Rs)
Vdm = sqrt.( halo.(Rs, P2[3], P2[4]) )
 plot( Rs, sqrt.(Vsum), lab = "Vtot", line = (3, :red) )
plot!( Rs, Vobs, yerr = errV, lab = "obs", line = (3, :gray), m = (3,:gray, 0.5) )
plot!( Rs, Vdm, lab = "DM", line = (2, :black) )
plot!( Rs, Vgas,  lab = "gas", line = (2, :blue) )
plot!( Rs, Vdisk, lab = "disk", line = (2, :green) )
plot!( Rs, Vbul,  lab = "bulge", line = (2, :orange), 
    legend = :right, xlab = "R, kpc", ylab = "V, km/s" )

In [ ]:
savefig("rot_curve_components_DM.png")

In [ ]:
plot()
for no in 150:159
    dat = Dat[no] # 14 80 110
    (Rs, Vobs, errV) = [dat[:,i] for i in 1:3 ]
    plot!( Rs, Vobs, ribbon = errV, line = 1, 
         lab = names[no][1:end-11] ) # 
end
plot!( legend = :right, xlab = "R, kpc", ylab = "V, km/s" )

In [ ]:
plot()
for no in 40:51
    dat = Dat[no] # 14 80 110
    (Rs, Vobs, errV) = [dat[:,i] for i in 1:3 ]
    plot!( Rs, Vobs, yerr = errV, line = 1, 
        m = (3, 0.5 ), lab = names[no][1:end-11] ) # 
end
plot!( legend = :right, xlab = "R, kpc", ylab = "V, km/s" )

In [ ]:
savefig("rot_curves.png")